In [4]:
import csv

file_path = '/home/ziczac/dev/position_control/motor_data1.csv'

rpm_a = []
rpm_b = []
position_a = []
position_b = []
timestamp = []

with open(file_path, mode='r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        rpm_a.append(float(row['RPM_Kanan']))
        rpm_b.append(float(row['RPM_Kiri']))
        position_a.append(float(row['Position_Kanan (deg)']))
        position_b.append(float(row['Position_Kiri (deg)']))
        # timestamp.append(float(row['Timestamp']))

print("RPM_A:", rpm_a)
print("RPM_B:", rpm_b)
print("Position_A:", position_a)
print("Position_B:", position_b)
# print("Timestamp:", timestamp)


RPM_A: [0.0, 1.0, 3.0, 6.0, 11.0, 15.0, 17.0, 18.0, 19.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 20.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 

In [5]:
import math
total_distance_a  = 0
total_distance_b = 0

for i in range(1, len(timestamp)):
        delta_t = (timestamp[i] - timestamp[i - 1])
        
        angular_displacement_a = rpm_a[i] * (2 * math.pi / 60) * delta_t
        angular_displacement_b = -rpm_b[i] * (2 * math.pi / 60) * delta_t
        
        distance_a = angular_displacement_a * 0.05
        distance_b = angular_displacement_b * 0.05
        
        total_distance_a += distance_a
        total_distance_b += distance_b

print("Total A : " , total_distance_a, "m")
print("Total B : " , total_distance_b, "m")

Total A :  0 m
Total B :  0 m


In [8]:
import numpy as np

def difference_angle(first, second):
    return (second - first + 180) % 360 - 180

dist_left = 0
dist_right = 0
prev_angle_left = None
prev_angle_right = None
for i in range(len(rpm_a)):
    if prev_angle_left is not None:
        d_theta_L = difference_angle(prev_angle_left, -position_a[i])
        dist_left += (np.pi * 2 * 0.05 * (d_theta_L / 360))
    prev_angle_left = -position_a[i]
    if prev_angle_right is not None:
        d_theta_R = difference_angle(prev_angle_right, position_b[i])
        dist_right += (np.pi * 2 * 0.05 * (d_theta_R / 360))
    prev_angle_right = position_b[i]

print("Total A : " , dist_left, "m")
print("Total B : " , dist_right, "m")

Total A :  1.1408293322447636 m
Total B :  1.1432933264828726 m
